# 201 Spark basics

The goal of this package is to get familiar with Spark programming.

- [Spark programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
- [PySpark RDD APIs](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)

## 201-5 Spark warm-up

Load the ```riddle``` dataset and try the following actions:
- Show its content (```collect```)
- Count the rows (```count```)
- Split phrases into words (```map``` or ```flatMap```; what’s the difference?)
- Check the results (remember: evaluation is lazy)

In [ ]:
bucketname = "univ-tours-bd2223-egallinucci"

rddRiddle = sc.textFile("s3a://"+bucketname+"/datasets/riddle.txt")

In [ ]:
rddRiddle.collect()

In [ ]:
rddRiddle.count()

In [ ]:
rddRiddle.map(lambda row: row.split(" ") ).collect()

In [ ]:
rddRiddle.flatMap(lambda row: row.split(" ") ).collect()

## 201-6 Spark jobs

Implement some basic Spark jobs. Try first on the ```riddle``` dataset, then execute on the ```fiction``` one.

- Jobs:
  - Count the number of occurrences of each word
    - Result: ('si', 1), ('ton', 2), ('tonton', 3), ...
  - Count the number of occurrences of words of given lengths
    - Result: (2, 1), (3, 3), ...
  - Count the average length of words given their first letter (hint: check the slides for aggregating multiple values)
    - Result: ('s', 3.0), ('m', 3.0), ('t', 4.7)
  - Return the inverted index of words (hints: use the ```zipWithIndex``` method; to print an key-value RDD with a list in the value, use ```mapValues(list)```)
    - Result: ('si', \[0]), ('ton', \[0, 1]), ...

- How is the output sorted? How can you sort by value?
- Try the ```toDebugString``` function to check the execution plans

In [ ]:
rddRiddle = sc.textFile("s3a://"+bucketname+"/datasets/riddle.txt")
rddFiction = sc.textFile("s3a://"+bucketname+"/datasets/fiction")

In [ ]:
myRdd = rddRiddle

In [ ]:
# Word count

rddWordCount = myRdd.\
    flatMap(lambda row: row.split(" ") ).\
    map(lambda word: (word,1)).\
    reduceByKey(lambda x,y: x+y)

rddWordCount.take(100)

In [ ]:
# Word length count

rddWLC = myRdd.\
    flatMap(lambda row: row.split(" ") ).\
    map(lambda word: (len(word),1)).\
    reduceByKey(lambda x,y: x+y)

rddWLC.take(100)

In [ ]:
# Average word length by initial

rddWLBA = myRdd.\
    flatMap(lambda row: row.split(" ") ).\
    filter(lambda word: len(word)>0).\
    map(lambda word: (word[0:1].lower(), (len(word), 1))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    mapValues(lambda v: v[0]/v[1])

rddWLBA.take(100)

In [ ]:
# Inverted index (word-based offset)

rddII = myRdd.\
    flatMap(lambda row: row.split(" ")).\
    filter(lambda word: len(word)>0).\
    zipWithIndex().\
    groupByKey().\
    mapValues(list)

rddII.take(100)

In [ ]:
# Inverted index (sentence-based offset)

rddII2 = myRdd.\
    zipWithIndex().\
    map(lambda el: (el[1], el[0])).\
    flatMapValues(lambda row: row.split(" ")).\
    filter(lambda el: len(el[1])>0).\
    map(lambda el: (el[1], el[0])).\
    distinct().\
    groupByKey().\
    mapValues(list)

rddII2.take(100)

In [ ]:
# Sort an RDD by key

rddWordCount.sortByKey().collect()

# Sort an RDD by value

rddWordCount.map(lambda el: (el[1], el[0])).sortByKey().collect()